Definitions of basic gates

In [ ]:
import numpy as np
pi = np.pi

up = np.array([1., 0.])
down = np.array([0., 1.])
state = np.kron(up, up)

CX = np.array([[1., 0., 0., 0.,], [0., 1., 0., 0.], [0., 0., 0., 1.], [0., 0., 1., 0.]])
I = np.eye(2)
X = np.array([[0., 1.], [1., 0.]])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])
CZ = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]])


In order to transpile a circuit, the following steps must be implemented:
1. Translate all gates into the native gateset
2. Choose a given ordering
    a. convert everything to swap gates
    b. minimize the number of swap gates
3. Find the best connectivity for this ordering
4. Repeat to find the best ordering and connectivity for a given algorithm

1. Translate all gates into the native gateset (here CZ, X90, Z)

In [37]:
# testing the equivalence library for the CNOT gate

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import CXGate, HGate, RYGate, RXGate, SwapGate
from qiskit.circuit.equivalence_library import EquivalenceLibrary
import numpy as np
pi = np.pi

#done: cnot, h, ry, rz, rx


#not done: xx, yy, zz, 

#to improve: swap, rx

# cnot

cnot = CXGate()

cx_eq = QuantumCircuit(2)
cx_eq.h(1)
cx_eq.cz(0, 1)
cx_eq.h(1) # I guess pass this first, then decompose the hadamard gates?

eq_lib = EquivalenceLibrary()

eq_lib.add_equivalence(cnot, cx_eq)

# h
h = HGate()

h_eq = QuantumCircuit(1)
h_eq.p(pi/2, 0) # not technically in our library; find additional gateset for this
h_eq.x(0)
h_eq.rz(pi/2, 0)
h_eq.x(0)

eq_lib.add_equivalence(h, h_eq)

# y
t = Parameter('t')
ry = RYGate(t)
y_eq = QuantumCircuit(1)
y_eq.x(0)
y_eq.rz(-t, 0)
y_eq.x(0)

eq_lib.add_equivalence(ry, y_eq)

# rx (must be a better way, we end up with X*Z*X, which can probably be expressed as another gate/phase.)
t = Parameter('t')
rx = RXGate(t)
x_eq = QuantumCircuit(1)
x_eq.ry(pi/2, 0)
x_eq.rz(t, 0)
x_eq.ry(pi/2, 0)

eq_lib.add_equivalence(rx, x_eq)

# swap (also must have better form for this gateset)
swap = SwapGate()
swap_eq = QuantumCircuit(2)
swap_eq.cx(0, 1)
swap_eq.cx(1, 0)
swap_eq.cx(0, 1)

eq_lib.add_equivalence(swap, swap_eq)


In [41]:
# testing new circuit

circ = QuantumCircuit(2)
# swap as cx
circ.cx(0, 1)
circ.cx(1, 0)
circ.cx(0, 1)

# transpile the circuit

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import BasisTranslator


basis_gates = ['cz', 'x', 'rz', 'p']
pm = PassManager()
pm.append(BasisTranslator(eq_lib, basis_gates))

# Step 5: Transpile the circuit
transpiled_qc = pm.run(circ)

# Draw the transpiled circuit
print(transpiled_qc.draw())

                                       ┌────────┐┌───┐┌─────────┐┌───┐   »
q_0: ────────────────────────────────■─┤ P(π/2) ├┤ X ├┤ Rz(π/2) ├┤ X ├─■─»
     ┌────────┐┌───┐┌─────────┐┌───┐ │ ├────────┤├───┤├─────────┤├───┤ │ »
q_1: ┤ P(π/2) ├┤ X ├┤ Rz(π/2) ├┤ X ├─■─┤ P(π/2) ├┤ X ├┤ Rz(π/2) ├┤ X ├─■─»
     └────────┘└───┘└─────────┘└───┘   └────────┘└───┘└─────────┘└───┘   »
«     ┌────────┐┌───┐┌─────────┐┌───┐                                  
«q_0: ┤ P(π/2) ├┤ X ├┤ Rz(π/2) ├┤ X ├─■────────────────────────────────
«     ├────────┤├───┤├─────────┤├───┤ │ ┌────────┐┌───┐┌─────────┐┌───┐
«q_1: ┤ P(π/2) ├┤ X ├┤ Rz(π/2) ├┤ X ├─■─┤ P(π/2) ├┤ X ├┤ Rz(π/2) ├┤ X ├
«     └────────┘└───┘└─────────┘└───┘   └────────┘└───┘└─────────┘└───┘
